## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import json

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

# Import WeatherPy_Database.csv

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,mto wa mbu,TZ,-3.3500,35.8500,72.30,84,95,3.44,light rain
1,1,ahipara,NZ,-35.1667,173.1667,66.65,96,35,3.06,scattered clouds
2,2,yulara,AU,-25.2406,130.9889,82.40,32,5,11.50,clear sky
3,3,faanui,PF,-16.4833,-151.7500,82.13,72,100,8.81,overcast clouds
4,4,hobyo,SO,5.3505,48.5268,86.92,54,81,12.75,broken clouds


# Get min and max user preferences

In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 95


# Filter and clean data

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isna().sum()

Unnamed: 0     0
City           0
Country        6
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(how='any') #inplace = True) 

In [7]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 267 entries, 0 to 690
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   267 non-null    int64  
 1   City         267 non-null    object 
 2   Country      267 non-null    object 
 3   Lat          267 non-null    float64
 4   Lng          267 non-null    float64
 5   Max Temp     267 non-null    float64
 6   Humidity     267 non-null    int64  
 7   Cloudiness   267 non-null    int64  
 8   Wind Speed   267 non-null    float64
 9   Description  267 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 22.9+ KB


# Create hotel_df

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()
hotel_df.columns = ["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]
# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)
#print(type(clean_df['City'][0]))

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,mto wa mbu,TZ,72.30,light rain,-3.3500,35.8500,
2,yulara,AU,82.40,clear sky,-25.2406,130.9889,
3,faanui,PF,82.13,overcast clouds,-16.4833,-151.7500,
4,hobyo,SO,86.92,broken clouds,5.3505,48.5268,
5,hithadhoo,MV,83.08,few clouds,-0.6000,73.0833,
6,praia,CV,71.60,clear sky,14.9215,-23.5087,
11,itarema,BR,76.01,broken clouds,-2.9248,-39.9167,
12,hermanus,ZA,73.99,overcast clouds,-34.4187,19.2345,
13,cape town,ZA,72.00,clear sky,-33.9258,18.4232,
14,maumere,ID,84.67,clear sky,-8.6199,122.2111,


# Find the nearest hotel for each preferred city

In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat=row['Lat']
    lng=row['Lng']
    city=row['City']
    country=row['Country']
    latlng = f"{lat},{lng}"
    
    params["location"] = latlng
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    #base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    response = requests.get(base_url, params=params).json()
   
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    #hotel_df['Hotel Name'] = hotel_json['results'][0]['name']
    #print(json.dumps(response, indent=4, sort_keys=True))
 
    print(f"Retrieving Results for Index {index}: {city}.")
    results = response['results']
    
    try:
        name = results[0]['name']
        print(f"Closest hotel in {city}, {country} is {name}.")
        #types_df.loc[index, 'name'] = results[0]['name']
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

print("Done")


Retrieving Results for Index 0: mto wa mbu.
Closest hotel in mto wa mbu, TZ is Fanaka Safaris Campsite & Lodges.
------------
Retrieving Results for Index 2: yulara.
Closest hotel in yulara, AU is Desert Gardens Hotel - Ayers Rock Resort.
------------
Retrieving Results for Index 3: faanui.
Closest hotel in faanui, PF is Bora-Bora Pearl Beach Resort & Spa.
------------
Retrieving Results for Index 4: hobyo.
Closest hotel in hobyo, SO is Halane Hotel and Restaurent.
------------
Retrieving Results for Index 5: hithadhoo.
Closest hotel in hithadhoo, MV is Scoop Guest House.
------------
Retrieving Results for Index 6: praia.
Closest hotel in praia, CV is LT Aparthotel.
------------
Retrieving Results for Index 11: itarema.
Closest hotel in itarema, BR is Sea Lounge! Bar, Club & Restaurant.
------------
Retrieving Results for Index 12: hermanus.
Closest hotel in hermanus, ZA is Misty Waves Boutique Hotel.
------------
Retrieving Results for Index 13: cape town.
Closest hotel in cape town,

Retrieving Results for Index 195: condega.
Closest hotel in condega, NI is Estancia Doña Rosario.
------------
Retrieving Results for Index 196: iranshahr.
Closest hotel in iranshahr, IR is پلکه بزرگ.
------------
Retrieving Results for Index 199: lalmohan.
Closest hotel in lalmohan, BD is Sundorbon courier service.
------------
Retrieving Results for Index 200: quatre cocos.
Closest hotel in quatre cocos, MU is Emeraude Beach Attitude Hotel.
------------
Retrieving Results for Index 201: tiarei.
Closest hotel in tiarei, PF is Le Rocher de Tahiti.
------------
Retrieving Results for Index 202: zhanakorgan.
Closest hotel in zhanakorgan, KZ is "Ақпалас" қонақ үйі.
------------
Retrieving Results for Index 203: cabo san lucas.
Closest hotel in cabo san lucas, MX is Hotel Tesoro Los Cabos.
------------
Retrieving Results for Index 207: barabai.
Closest hotel in barabai, ID is HOTEL BOURAQ INDAH PERMAI.
------------
Retrieving Results for Index 208: madimba.
Missing field/result... skipping

Retrieving Results for Index 401: adrar.
Missing field/result... skipping.
------------
Retrieving Results for Index 402: biltine.
Closest hotel in biltine, TD is Hôtel Trois Étoiles de Biltine.
------------
Retrieving Results for Index 405: brokopondo.
Closest hotel in brokopondo, SR is Matu Island.
------------
Retrieving Results for Index 409: kassala.
Closest hotel in kassala, SD is Amassi Hotel Gash.
------------
Retrieving Results for Index 412: paratebueno.
Closest hotel in paratebueno, CO is Casa Hotel Paratebueno.
------------
Retrieving Results for Index 413: porto belo.
Closest hotel in porto belo, BR is Costa Verde Beach Hotel.
------------
Retrieving Results for Index 420: natal.
Closest hotel in natal, BR is Villa Park Hotel | Natal.
------------
Retrieving Results for Index 425: namatanai.
Closest hotel in namatanai, PG is Namatanai Hotel.
------------
Retrieving Results for Index 426: monrovia.
Closest hotel in monrovia, LR is Sunset Inn.
------------
Retrieving Results

Retrieving Results for Index 565: zastron.
Closest hotel in zastron, ZA is Highlands Guest House.
------------
Retrieving Results for Index 567: kruisfontein.
Closest hotel in kruisfontein, ZA is Oyster Bay House Rental.
------------
Retrieving Results for Index 571: vao.
Closest hotel in vao, NC is Hotel Kou-Bugny.
------------
Retrieving Results for Index 575: mahanoro.
Closest hotel in mahanoro, MG is BUNGALOW MAHAMANINA.
------------
Retrieving Results for Index 577: lantawan.
Closest hotel in lantawan, PH is Durano Ecofarm and Spring Resort.
------------
Retrieving Results for Index 583: soure.
Closest hotel in soure, BR is Casarão.
------------
Retrieving Results for Index 587: masuguru.
Missing field/result... skipping.
------------
Retrieving Results for Index 592: singaparna.
Closest hotel in singaparna, ID is Wisma Dewi Hotel.
------------
Retrieving Results for Index 597: malur.
Closest hotel in malur, IN is RG Hotels and Resort.
------------
Retrieving Results for Index 603

In [10]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,mto wa mbu,TZ,72.30,light rain,-3.3500,35.8500,Fanaka Safaris Campsite & Lodges
2,yulara,AU,82.40,clear sky,-25.2406,130.9889,Desert Gardens Hotel - Ayers Rock Resort
3,faanui,PF,82.13,overcast clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
4,hobyo,SO,86.92,broken clouds,5.3505,48.5268,Halane Hotel and Restaurent
5,hithadhoo,MV,83.08,few clouds,-0.6000,73.0833,Scoop Guest House


# Drop any rows with empty hotel name

In [11]:
# 7. Drop the rows where there is no Hotel Name.
f_has_hotel = hotel_df["Hotel Name"] != ""
hotel_df = hotel_df.loc[ f_has_hotel , : ]
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,mto wa mbu,TZ,72.30,light rain,-3.3500,35.8500,Fanaka Safaris Campsite & Lodges
2,yulara,AU,82.40,clear sky,-25.2406,130.9889,Desert Gardens Hotel - Ayers Rock Resort
3,faanui,PF,82.13,overcast clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
4,hobyo,SO,86.92,broken clouds,5.3505,48.5268,Halane Hotel and Restaurent
5,hithadhoo,MV,83.08,few clouds,-0.6000,73.0833,Scoop Guest House


#  Write CSV file

In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

# Create template for marker layer

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

# Create map with markers

In [17]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))